# to do
- test set has to be put in a goof form
- the promointerval was taken out until I can put it in a good form as well
- how to change the arrays inside an array as individual columns (otherwise I think SVM etc will not work)

In [19]:
import numpy as np
import pandas 
import sklearn as sk
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from math import sqrt


In [2]:
#read the files
store = pandas.read_csv('store.csv', sep = ',')
train_initial = pandas.read_csv('train_v2.csv',dtype={"StateHoliday": str})
test = pandas.read_csv('test_v2.csv', sep = ",") 

#take out the sales (we don't have this info in the test)
sales_train = train_initial['Sales']
train = train_initial.drop('Sales', axis=1)


#take out days were it's closed - When 0 customers will be identified in test data
# we will automatically put the test sales at 0.
sales_train = pandas.Series.to_frame(sales_train)
train = train[train.Customers != 0]
sales_train = sales_train[sales_train.Sales != 0]

In [3]:
# put the 2 types of data together
df = train.join(store.set_index('Store'), on='Store')
test = test.join(store.set_index('Store'), on='Store')

# extract the date as day/month/year/weak of the year
df['Date']  = pandas.to_datetime(df['Date'], errors='coerce')
df['Year']  = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['WeekOfYear'] = df['Date'].dt.weekofyear

test['Date']  = pandas.to_datetime(test['Date'], errors='coerce')
test['Year']  = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.weekofyear

In [4]:
# combine train and test (in that oder) to make sure all tranformation are the same for 
# test and train (same dummies etc)
number_train = df.shape[0]
number_test = test.shape[0]
frames = [df,test]
result = pandas.concat(frames)

In [5]:
result = result.fillna(-1)

In [6]:
# the code in this cell has been inspired by a similar function foud online at :
# http://www.akbarian.org/notes/sale-forecasting-time-series-analysis/

#we will detect wether there is a promotion in this month:
monthAsString = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
                     7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
result['MonthCompare'] = result.Month.map(monthAsString)

result['IsPromo2Month'] = 0

for interval in result.PromoInterval.unique():
        if interval != -1:
            for month in interval.split(','):
                condition = (result.MonthCompare == month) & (result.PromoInterval == interval)
                # If promo started this year, Week of Year must be > Promo2SinceWeek
                cond1 = (condition & (result.Year == result.Promo2SinceYear)
                         & (result.WeekOfYear >= result.Promo2SinceWeek) )
                # Or If promo started previous year, Week of Year doesn't matter
                cond2 = condition & (result.Year > result.Promo2SinceYear)
                final_condition = cond1 | cond2
                result.loc[final_condition, 'IsPromo2Month'] = 1


# detect if there is competition open at the time of the evaluation
result['Competition'] = 0
first_condition = result.Year > result.CompetitionOpenSinceYear
second_condition = ((result.Year == result.CompetitionOpenSinceYear)
          & (result.Month >= result.CompetitionOpenSinceMonth))
fullcond = first_condition | second_condition
result.loc[fullcond, 'Competition'] = 1

In [7]:
# replace the caracters by numbers, to be used in decision trees later
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
result.StateHoliday.replace(mappings, inplace=True)
result.StoreType.replace(mappings, inplace=True)
result.Assortment.replace(mappings, inplace=True)

In [8]:
#drop data that is not important anymore
result.drop(['MonthCompare','PromoInterval','Date'], axis=1, inplace=True)

In [9]:
# By analizing the data some funny behaviour can happen in the beginning or the end of every month
# to adress this issue we add 2 more parameters to identify the beginning and end of month
# finally in general the month of december behaves very differently than the others
# because of christmas.

result['Friday'] = 0
result['EarlyInMonth'] = 0
result['LateInMonth'] = 0
result['December'] = 0
result.loc[result.DayOfWeek == 5,'Friday'] = 1
result.loc[result.Day < 5, 'EarlyInMonth'] = 1
result.loc[result.Day > 25, 'LateInMonth'] = 1
result.loc[result.Month == 12,'December'] = 1

In [10]:
# redivide the result dataframe in the testing and the training sets
df = result.iloc[:number_train,:]
test = result.iloc[number_train:,:]

In [11]:
# conversion everything to float32 to have uniform format.
df = df.astype(np.float32)
test = test.astype(np.float32)

In [12]:
#convert a numpy array
data_train = df.values
sales_train = pandas.DataFrame.as_matrix(sales_train)
sales_train = sales_train.reshape(len(sales_train),)
#make a copy of the test data in a numoy array format
testing = test.values

In [13]:
X = data_train
y = sales_train

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.012)

In [15]:
randomforest = RandomForestRegressor(n_estimators=100, max_features = 'auto')
randomforest.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [16]:
def rmspe(y,y_pred):
    summ = 0
    for i in range(len(y)):
        if y[i] != 0:
            summ += (1-y_pred[i]/y[i])**2
    return sqrt(summ/len(y))

In [17]:
prediction_of_sales = randomforest.predict(testing)

for i in range (number_test) :
    if test['Customers'].iloc[i] == 0 :
        prediction_of_sales[i] = 0

In [20]:
prediction_in_sample = randomforest.predict(X_train)
prediction_validation = randomforest.predict(X_valid)

print ('error in samplle : %f', rmspe (y_train, prediction_in_sample))
print ('validation error : %f', rmspe (y_valid, prediction_validation))

error in samplle : %f 0.036298478992160974
validation error : %f 0.09585530128367901


In [21]:
submission_random_forest = pandas.DataFrame()
submission_random_forest['Sales'] = prediction_of_sales
# submission.Sales = submission.Sales.astype(int)
cols = ['Id','Sales']
submission_random_forest['Id'] = submission_random_forest.index + 1
submission_random_forest = submission_random_forest[cols]
submission_random_forest.to_csv('submission_random_forest.csv', index=False)

0.2246684265967324   10
0.19790389121535615  50
0.19413401753256496  100
0.19102393447665608  250
0.1903307154295898   500
0.19031775372145007  1000
0.19000211033516165  50000
0.19000798708563635  10000

#-------------


In [ ]:
auto, nfeatures = 100, and all extra data taken out : result = 0.3758

auto, nfeatures = 100 and kept some more features : result = 0.37458